# Enunciado:
A partir do dataset enviado, responda:

[Dataset](https://www.kaggle.com/datasets/kemical/kickstarter-projects)

### Importação e Análise Inicial
1) Importe o arquivo CSV.(0,33)
2) Quantas linhas tem o dataset?(0,33)
3) Quantas colunas tem o dataset?(0,33)
4) Quais colunas têm o tipo object?(0,34)
5) Quais colunas têm o tipo int64?(0,34)
6) Existe alguma coluna de outro tipo?(0,33)

### Análise Estatística
7) Qual foi o total, o máximo, o mínimo do objetivo financeiro de todos os projetos (goal)?(0,66)
8) Qual o total, o máximo, o mínimo e o valor que representa 25% do dinheiro levantado (pledged)?(0,67)
9) Qual o total, o máximo, o mínimo e o valor que representa 75% dos apoiadores dos projetos(backers)?(0,67)

### Análise de Categorias
10) Qual a category mais presente? E quantos valores únicos ela possui?(0,5)
11) Qual a main_category mais presente? E quantos valores únicos ela possui?(0,5)
12) Qual a currency mais presente? E quantos valores únicos ela possui?(0,5)
13) Execute o processo de dummy na coluna country.(0,5)

### Análise de Valores Nulos
14) Qual a quantidade de valores nulos em cada feature?(0,66)
15) Qual a proporção de valores nulos por valores não nulos em cada feature?(0,66)
16) Faça o tratamento desses nulos e explique por que escolheu cada método.(0,68)

### Visualização Gráfica
17) Visualização Gráfica da feature state.(2)

## Dependências e imports

In [ ]:
%pip install kagglehub==0.3.13
%pip install mlflow==3.5.1
%pip install pandas==2.3.3

import kagglehub
import mlflow
import pandas as pd
import os


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Constants

In [ ]:
KAGGLE_HUB_DATASET="kemical/kickstarter-projects"
MLFLOW_EXPERIMENT_NAME="etl_public_dataset"
MLFLOW_RUN_NAME="jupyter_run"
ARTIFACTS_PATH = "artifacts"

## MLFLOW - Configuração e utilidades

In [ ]:
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
mlflow.start_run(run_name=MLFLOW_RUN_NAME)

os.makedirs(ARTIFACTS_PATH, exist_ok=True)  # cria se não existir

def log_artifact(df: pd.DataFrame, name: str):
    path = os.path.join(ARTIFACTS_PATH, f"{name}.parquet")
    df.to_parquet(path, index=False)
    mlflow.log_artifact(path, artifact_path="data")

## Baixa e carrega o dataset

In [ ]:
mlflow.log_param("dataset", KAGGLE_HUB_DATASET)
dataset_path = kagglehub.dataset_download(KAGGLE_HUB_DATASET) # Exercício 1
print("Path do dataset:", dataset_path)
print("Arquivos do dataset:", os.listdir(dataset_path))
dataset_csv_path = os.path.join(dataset_path, os.listdir(dataset_path)[0])
print("Path do csv do dataset:", dataset_csv_path)

df_raw = pd.read_csv(dataset_csv_path, engine="python")
df_raw.head()

log_artifact(df_raw, "raw")

Path do dataset: /Users/vand/.cache/kagglehub/datasets/kemical/kickstarter-projects/versions/7
Arquivos do dataset: ['ks-projects-201801.csv', 'ks-projects-201612.csv']
Path do csv do dataset: /Users/vand/.cache/kagglehub/datasets/kemical/kickstarter-projects/versions/7/ks-projects-201801.csv


## Análise inicial

In [ ]:
df_raw.info()
df_raw.shape

# Exercício 2 -> 378662 linhas
# Exercício 3 -> 16 colunas

df_raw.dtypes.value_counts()
# Exercício 4 -> 8 objects
# Exercício 5 -> 2 int64
# Exercício 6 -> Sim, 5 float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


object     8
float64    5
int64      2
Name: count, dtype: int64

### Análise Estatística

In [ ]:
total_goal = sum(df_raw["goal"])
min_goal = min(df_raw["goal"])
max_goal = max(df_raw["goal"])
print(f"goal -> total: {total_goal}", f"min: {min_goal}", f"min: {max_goal}") # Exercício 7 -> 18584981597.970005, 0.01, 100000000.0

total_pledged = sum(df_raw["pledged"])
min_pledged = min(df_raw["pledged"])
max_pledged = max(df_raw["pledged"])
p25_pledged = df_raw["pledged"].quantile(0.25)
print(f"pledged -> total: {total_pledged}", f"min: {min_pledged}", f"max: {max_pledged}", f"p25: {p25_pledged}") # Exercício 8 -> 3666566639.660111, 0.0, 20338986.27, 30.0

total_backers = sum(df_raw["backers"])
min_backers = min(df_raw["backers"])
max_backers = max(df_raw["backers"])
p75_backers = df_raw["backers"].quantile(0.75)
print(f"backers-> total : {total_backers}", f"min: {min_backers}", f"max: {max_backers}", f"p75: {p75_backers}") # Exercício 9 -> 39993219, 0, 219382, 56.0


goal -> total: 18584981597.970005 min: 0.01 min: 100000000.0
pledged -> total: 3666566639.660111 min: 0.0 max: 20338986.27 p25: 30.0
backers-> total : 39993219 min: 0 max: 219382 p75: 56.0


### Análise de Categorias

In [ ]:
category_mode = df_raw["category"].mode()[0]
category_unique_len = len(df_raw["category"].unique())
print("category ->", f"mode: {category_mode}", f"unique len: {category_unique_len}") # Exercício 10 -> Product Design, 159

main_category_mode = df_raw["main_category"].mode()[0]
main_category_unique_len = len(df_raw["main_category"].unique())
print("main_category ->", f"mode: {main_category_mode}", f"unique len: {main_category_unique_len}") # Exercício 11 -> Film & Video, 15

currency_mode = df_raw["currency"].mode()[0]
currency_unique_len = len(df_raw["currency"].unique())
print("currency ->", f"mode: {currency_mode}", f"unique len: {currency_unique_len}") # Exercício 12 -> USD, 14

df_country_dummie = pd.get_dummies(df_raw["country"]) # Exercício 13 -> Vide info
df_country_dummie.info()


category -> mode: Product Design unique len: 159
main_category -> mode: Film & Video unique len: 15
currency -> mode: USD unique len: 14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 23 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   AT      378661 non-null  bool 
 1   AU      378661 non-null  bool 
 2   BE      378661 non-null  bool 
 3   CA      378661 non-null  bool 
 4   CH      378661 non-null  bool 
 5   DE      378661 non-null  bool 
 6   DK      378661 non-null  bool 
 7   ES      378661 non-null  bool 
 8   FR      378661 non-null  bool 
 9   GB      378661 non-null  bool 
 10  HK      378661 non-null  bool 
 11  IE      378661 non-null  bool 
 12  IT      378661 non-null  bool 
 13  JP      378661 non-null  bool 
 14  LU      378661 non-null  bool 
 15  MX      378661 non-null  bool 
 16  N,0"    378661 non-null  bool 
 17  NL      378661 non-null  bool 
 18  NO      378661 non-null  bo

### Análise de Valores Nulos

In [ ]:
print(df_raw.isnull().sum().sort_values(ascending=False)) # Exercício 15 -> name == 4, usd pledged == 3797

nulls = df_raw.isnull().sum()
total = len(df_raw)
not_nulls = total - nulls

proportion = (nulls / not_nulls).round(3)
print(f"proportion of nulls:\n{proportion}")  # Exercício 16

# Exercício 17
df_clear = df_raw.copy()

df_clear = df_clear.dropna(subset=["name"]) # Removendo os com nome null, pois sem nome não faz sentido analistar o projeto
mlflow.log_param("remove-null-name", True)

usd_pledged_median = df_clear["usd pledged"].median() # Imputando a mediana pois as nulls representam apenas 0.01% da base
print(f"usd pledged median: {usd_pledged_median}")
mlflow.log_param("imput-usd pledged", "median")

df_clear = df_clear.fillna(usd_pledged_median)

display(df_clear.head())
print(df_clear.isnull().sum().sort_values(ascending=False))
log_artifact(df_clear, "non-null")


usd pledged         3797
name                   4
ID                     0
main_category          0
currency               0
deadline               0
category               0
goal                   0
launched               0
state                  0
pledged                0
backers                0
country                0
usd_pledged_real       0
usd_goal_real          0
dtype: int64
proportion of nulls:
ID                  0.00
name                0.00
category            0.00
main_category       0.00
currency            0.00
deadline            0.00
goal                0.00
launched            0.00
pledged             0.00
state               0.00
backers             0.00
country             0.00
usd pledged         0.01
usd_pledged_real    0.00
usd_goal_real       0.00
dtype: float64
usd pledged median: 394.765


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


ID                  0
name                0
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
usd_goal_real       0
dtype: int64


## Finaliza o experimento

In [ ]:
mlflow.end_run()